In [1]:
from tools.data import get_datasets

train, test = get_datasets()

Reading data file


In [2]:
%matplotlib inline

import matplotlib.pyplot as plt

In [3]:
# Classification -1 vs >=0
# on prépare les données:
# tout ce qui est un chiffre -> 1
# tout ce qui est du bruit -> 0
import numpy as np

train[train[:,-1] >= 0,-1] =  0
test[test[:,-1] >= 0,-1] =  0

train[:,-1] = train[:,-1] + 1
test[:,-1] = test[:,-1] + 1

In [16]:
print (train[:,-1] == 0).sum()
print (train[:,-1] == 1).sum()

train_0 = np.compress(train[:,-1] == 0, train, axis=0)
train_1 = np.compress(train[:,-1] == 1, train, axis=0)[:train_0.shape[0]]

train_balanced = np.concatenate((train_0, train_1))

print train_balanced.shape

indices = np.arange(train_0.shape[0]*2)
np.random.shuffle(indices)

train_balanced = train_balanced[indices, :]

print (train_balanced[:,-1] == 0).sum()
print (train_balanced[:,-1] == 1).sum()

8173
21827
(16346, 2050)
8173
8173


In [14]:
import theano.tensor as T
import theano

y_true = T.tensor3()
y_pred = T.tensor3()

nll = theano.function([y_true, y_pred], T.log(T.tensordot(y_true, y_pred)))


In [15]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Merge, Flatten
from keras.optimizers import SGD, Adadelta
from keras.layers.convolutional import Convolution1D, MaxPooling1D

nb_filters = 10
filter_length = 5

first_layers = []
for i in range(4):
    first_layers.append(Sequential())
    first_layers[-1].add(Convolution1D(nb_filters, filter_length,
                        border_mode='valid',
                        input_shape=(408, 1), init='glorot_normal'))
    first_layers[-1].add(MaxPooling1D(pool_length=2))
    first_layers[-1].add(Activation('tanh'))
    first_layers[-1].add(Convolution1D(nb_filters, filter_length,
                        border_mode='valid', init='glorot_normal'))
    first_layers[-1].add(MaxPooling1D(pool_length=2))
    first_layers[-1].add(Activation('tanh'))
    first_layers[-1].add(Flatten())


model = Sequential()
model.add(Merge(first_layers, mode='concat', concat_axis=1))

model.add(Dense(100))
model.add(Activation('tanh'))
model.add(Dense(2))
model.add(Activation('softmax'))

adadelta = Adadelta()
model.compile(loss=nll, optimizer=adadelta)

TypeError: 'TensorVariable' object is not callable

In [18]:
from keras.utils import np_utils

train_x = train_balanced.astype('float32')
valid_x = test[:5000].astype('float32')
test_x = test[5000:].astype('float32')

train_y = np_utils.to_categorical(train_balanced[:,-1], 2)
valid_y = np_utils.to_categorical(test[:5000,-1], 2)
test_y = np_utils.to_categorical(test[5000:,-1], 2)

train_4 = [train_x[:,1+512*0:1+512*0+408,None], train_x[:,1+512*1:1+512*1+408,None], train_x[:,1+512*2:1+512*2+408,None], train_x[:,1+512*3:1+512*3+408,None]]
valid_4 = [valid_x[:,1+512*0:1+512*0+408,None], valid_x[:,1+512*1:1+512*1+408,None], valid_x[:,1+512*2:1+512*2+408,None], valid_x[:,1+512*3:1+512*3+408,None]]
test_4 = [test_x[:,1+512*0:1+512*0+408,None], test_x[:,1+512*1:1+512*1+408,None], test_x[:,1+512*2:1+512*2+408,None], test_x[:,1+512*3:1+512*3+408,None]]


In [ ]:
batch_size = 250
nb_epoch = 100

model.fit(train_4, train_y,
          batch_size=batch_size, nb_epoch=nb_epoch,
          show_accuracy=True, verbose=2,
          validation_data=(valid_4, valid_y))
score = model.evaluate(test_4, test_y,
                       show_accuracy=True, verbose=0)
print score

Train on 16346 samples, validate on 5000 samples
Epoch 1/100
34s - loss: 1.2898 - acc: 0.5022 - val_loss: 0.7972 - val_acc: 0.2616
Epoch 2/100
34s - loss: 0.6965 - acc: 0.4985 - val_loss: 0.7408 - val_acc: 0.2616


In [47]:
print score
model.fit(train_4, train_y,
          batch_size=batch_size, nb_epoch=nb_epoch,
          show_accuracy=True, verbose=2,
          validation_data=(valid_4, valid_y))
score = model.evaluate(test_4, test_y,
                       show_accuracy=True, verbose=0)

[0.61743914956494994, 0.73006852749456796]
Train on 30000 samples, validate on 5000 samples
Epoch 1/100
32s - loss: 0.6286 - acc: 0.7186 - val_loss: 0.6145 - val_acc: 0.7396
Epoch 2/100
32s - loss: 0.6015 - acc: 0.7253 - val_loss: 0.5728 - val_acc: 0.7408
Epoch 3/100
32s - loss: 0.6172 - acc: 0.7217 - val_loss: 0.5834 - val_acc: 0.7404
Epoch 4/100
32s - loss: 0.6166 - acc: 0.7224 - val_loss: 0.5868 - val_acc: 0.7408
Epoch 5/100
32s - loss: 0.6079 - acc: 0.7302 - val_loss: 0.5746 - val_acc: 0.7408
Epoch 6/100
32s - loss: 0.5953 - acc: 0.7304 - val_loss: 0.5898 - val_acc: 0.7408
Epoch 7/100
32s - loss: 0.6254 - acc: 0.7178 - val_loss: 1.5663 - val_acc: 0.7384
Epoch 8/100
32s - loss: 1.2727 - acc: 0.5930 - val_loss: 0.8187 - val_acc: 0.7386
Epoch 9/100
32s - loss: 1.3237 - acc: 0.5839 - val_loss: 0.5947 - val_acc: 0.7392
Epoch 10/100
33s - loss: 1.2637 - acc: 0.6048 - val_loss: 2.3780 - val_acc: 0.7384
Epoch 11/100
32s - loss: 1.3935 - acc: 0.5946 - val_loss: 0.8742 - val_acc: 0.2614
Epoc

KeyboardInterrupt: 

In [ ]:
# ca c'est le résultat qu'on obtient en prenant un classifieur
# qui prédit tout le temps 1 (classe majoritaire)
print float(np.sum(test[:,-1]==1))/test.shape[0]